# 🔬 Microscope Image Defect Detection - Simple Notebook

This notebook detects defects in microscope images.

**What it finds:**
- Scratches (long thin defects)
- Voids (dark holes)
- Particles (bright spots)
- Edge problems
- Texture issues

**Just run each cell in order!**

## Step 1: Install Required Packages

Run this cell once to install everything you need.

In [ ]:
# Install packages (run once)
!pip install opencv-python numpy matplotlib pillow

print("✓ Installation complete!")

## Step 2: Import Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

# Make plots appear in notebook
%matplotlib inline

# Make plots bigger
plt.rcParams['figure.figsize'] = [15, 10]

print("✓ Libraries imported!")

## Step 3: Define Simple Defect Detector

This is the detection code - lots of comments to explain what it does!

In [ ]:
class SimpleDefectDetector:
    """
    Simple defect detector for microscope images.
    
    Finds 4 types of defects using different methods.
    """
    
    def __init__(self, min_size=10, max_size=50000):
        """
        Create detector.
        
        min_size: Smallest defect to find (pixels)
        max_size: Largest defect to find (pixels)
        """
        self.min_size = min_size
        self.max_size = max_size
        print(f"Detector created! Finding defects between {min_size} and {max_size} pixels.")
    
    
    def detect(self, image):
        """
        Main function to find defects.
        
        image: Your microscope image (color or grayscale)
        
        Returns: List of defects found
        """
        print("Starting detection...")
        
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        defects = []
        
        # Method 1: Find bright/dark spots
        print("  Finding bright/dark spots...")
        defects.extend(self._find_spots(gray))
        
        # Method 2: Find scratches
        print("  Finding scratches...")
        defects.extend(self._find_scratches(gray))
        
        # Method 3: Find voids and particles
        print("  Finding voids and particles...")
        defects.extend(self._find_voids_particles(gray))
        
        # Remove duplicates
        defects = self._remove_duplicates(defects)
        
        print(f"✓ Found {len(defects)} defects total!")
        return defects
    
    
    def _find_spots(self, gray):
        """Find bright and dark spots"""
        defects = []
        
        # Reduce noise
        denoised = cv2.bilateralFilter(gray, 9, 75, 75)
        
        # Adaptive threshold
        thresh = cv2.adaptiveThreshold(denoised, 255, 
                                      cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                      cv2.THRESH_BINARY_INV, 11, 2)
        
        # Clean up
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel, iterations=1)
        
        # Find shapes
        contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if self.min_size <= area <= self.max_size:
                x, y, w, h = cv2.boundingRect(cnt)
                defects.append({
                    'type': 'spot',
                    'bbox': (x, y, w, h),
                    'area': area,
                    'contour': cnt
                })
        
        return defects
    
    
    def _find_scratches(self, gray):
        """Find scratches using edge detection"""
        defects = []
        
        # Blur and find edges
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        
        # Connect nearby edges
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        dilated = cv2.dilate(edges, kernel, iterations=2)
        
        # Find shapes
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area >= self.min_size:
                x, y, w, h = cv2.boundingRect(cnt)
                
                # Check if elongated (scratch)
                aspect_ratio = max(w, h) / (min(w, h) + 0.001)
                
                if aspect_ratio > 2:
                    defects.append({
                        'type': 'scratch',
                        'bbox': (x, y, w, h),
                        'area': area,
                        'contour': cnt
                    })
        
        return defects
    
    
    def _find_voids_particles(self, gray):
        """Find voids (dark) and particles (bright)"""
        defects = []
        
        # Morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
        tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, kernel)
        blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
        
        # Threshold
        _, tophat_thresh = cv2.threshold(tophat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        _, blackhat_thresh = cv2.threshold(blackhat, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Combine
        combined = cv2.bitwise_or(tophat_thresh, blackhat_thresh)
        
        # Find shapes
        contours, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if self.min_size <= area <= self.max_size:
                x, y, w, h = cv2.boundingRect(cnt)
                
                # Check if void or particle
                mask = np.zeros(gray.shape, dtype=np.uint8)
                cv2.drawContours(mask, [cnt], -1, 255, -1)
                mean_val = cv2.mean(gray, mask=mask)[0]
                
                dtype = 'void' if mean_val < np.mean(gray) else 'particle'
                
                defects.append({
                    'type': dtype,
                    'bbox': (x, y, w, h),
                    'area': area,
                    'contour': cnt
                })
        
        return defects
    
    
    def _remove_duplicates(self, defects):
        """Remove overlapping detections"""
        if not defects:
            return []
        
        # Sort by area (largest first)
        defects = sorted(defects, key=lambda d: d['area'], reverse=True)
        
        keep = []
        for defect in defects:
            # Check overlap with kept defects
            overlap = False
            for kept in keep:
                if self._calculate_overlap(defect['bbox'], kept['bbox']) > 0.3:
                    overlap = True
                    break
            
            if not overlap:
                keep.append(defect)
        
        return keep
    
    
    def _calculate_overlap(self, bbox1, bbox2):
        """Calculate IoU (Intersection over Union)"""
        x1, y1, w1, h1 = bbox1
        x2, y2, w2, h2 = bbox2
        
        # Find intersection
        x_left = max(x1, x2)
        y_top = max(y1, y2)
        x_right = min(x1 + w1, x2 + w2)
        y_bottom = min(y1 + h1, y2 + h2)
        
        if x_right < x_left or y_bottom < y_top:
            return 0.0
        
        intersection = (x_right - x_left) * (y_bottom - y_top)
        area1 = w1 * h1
        area2 = w2 * h2
        union = area1 + area2 - intersection
        
        return intersection / union if union > 0 else 0.0


print("✓ Detector class defined!")

## Step 4: Helper Functions for Visualization

In [ ]:
def draw_defects(image, defects):
    """
    Draw boxes around defects.
    
    Returns: Image with defects highlighted
    """
    # Make a copy
    result = image.copy()
    
    # Colors for different types
    colors = {
        'scratch': (255, 0, 0),      # Red
        'void': (0, 0, 255),         # Blue
        'particle': (0, 255, 0),     # Green
        'spot': (255, 255, 0),       # Cyan
    }
    
    # Draw each defect
    for defect in defects:
        x, y, w, h = defect['bbox']
        color = colors.get(defect['type'], (255, 255, 255))
        
        # Draw rectangle
        cv2.rectangle(result, (x, y), (x+w, y+h), color, 2)
        
        # Add label
        label = f"{defect['type']}"
        cv2.putText(result, label, (x, y-5), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    
    # Add count
    text = f"Defects: {len(defects)}"
    cv2.putText(result, text, (10, 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    return result


def show_results(original, defects):
    """
    Display original and annotated images side by side.
    """
    # Draw defects
    annotated = draw_defects(original, defects)
    
    # Convert BGR to RGB for display
    if len(original.shape) == 3:
        original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
        annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    else:
        original_rgb = original
        annotated_rgb = annotated
    
    # Create figure
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    
    # Original
    axes[0].imshow(original_rgb, cmap='gray')
    axes[0].set_title('Original Image', fontsize=16)
    axes[0].axis('off')
    
    # Annotated
    axes[1].imshow(annotated_rgb)
    axes[1].set_title(f'Defects Found: {len(defects)}', fontsize=16)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print("\n" + "="*50)
    print("DEFECT SUMMARY")
    print("="*50)
    print(f"Total defects found: {len(defects)}\n")
    
    # Count by type
    type_counts = {}
    for d in defects:
        dtype = d['type']
        type_counts[dtype] = type_counts.get(dtype, 0) + 1
    
    print("By type:")
    for dtype, count in sorted(type_counts.items()):
        print(f"  {dtype}: {count}")
    
    print("\nDefect details:")
    for i, d in enumerate(defects, 1):
        x, y, w, h = d['bbox']
        print(f"  {i}. {d['type']} - Position: ({x},{y}), Size: {w}x{h}, Area: {d['area']:.0f}px²")


print("✓ Helper functions defined!")

## Step 5: Create a Test Image (No real image needed!)

This creates a synthetic microscope image with fake defects for testing.

In [ ]:
def create_test_image(width=800, height=600):
    """
    Create a fake microscope image with defects.
    """
    # Gray background
    img = np.ones((height, width), dtype=np.uint8) * 200
    
    # Add noise (texture)
    noise = np.random.normal(0, 10, (height, width))
    img = np.clip(img + noise, 0, 255).astype(np.uint8)
    
    # Add defects
    
    # 1. Scratch
    cv2.line(img, (100, 150), (700, 200), 80, 3)
    
    # 2. Void (dark spot)
    cv2.circle(img, (400, 300), 25, 50, -1)
    
    # 3. Particle (bright spot)
    cv2.circle(img, (250, 450), 15, 255, -1)
    
    # 4. More scratches
    cv2.line(img, (150, 500), (400, 520), 70, 2)
    cv2.line(img, (500, 100), (550, 250), 75, 2)
    
    # 5. Particle cluster
    cv2.circle(img, (550, 150), 8, 245, -1)
    cv2.circle(img, (565, 155), 6, 250, -1)
    cv2.circle(img, (555, 165), 7, 240, -1)
    
    # 6. Void cluster
    cv2.circle(img, (200, 250), 12, 60, -1)
    cv2.circle(img, (220, 255), 10, 55, -1)
    
    # Slight blur (simulate microscope)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    
    # Convert to BGR for consistency
    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    return img_bgr


# Create test image
test_image = create_test_image()

# Show it
plt.figure(figsize=(10, 7))
plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
plt.title('Test Microscope Image (Synthetic)', fontsize=16)
plt.axis('off')
plt.show()

print("✓ Test image created!")

## Step 6: Run Detection! 🚀

This is where the magic happens!

In [ ]:
# Create detector
detector = SimpleDefectDetector(min_size=10, max_size=50000)

# Detect defects
defects = detector.detect(test_image)

# Show results
show_results(test_image, defects)

## Step 7: Try with Your Own Image

Upload your own microscope image and run this cell!

In [ ]:
# ============================================
# OPTION 1: Load from file path
# ============================================

# Change this to your image path
image_path = "your_image.jpg"  # ← Change this!

# Load image
if os.path.exists(image_path):
    my_image = cv2.imread(image_path)
    
    if my_image is not None:
        print(f"✓ Loaded image: {image_path}")
        print(f"  Size: {my_image.shape[1]}x{my_image.shape[0]} pixels")
        
        # Detect defects
        my_defects = detector.detect(my_image)
        
        # Show results
        show_results(my_image, my_defects)
        
        # Save result
        result_image = draw_defects(my_image, my_defects)
        output_path = "result_" + os.path.basename(image_path)
        cv2.imwrite(output_path, result_image)
        print(f"\n✓ Saved result to: {output_path}")
    else:
        print(f"❌ Could not load image: {image_path}")
else:
    print(f"❌ File not found: {image_path}")
    print("\nTo use your own image:")
    print("1. Upload your image to the same folder as this notebook")
    print("2. Change 'your_image.jpg' above to your filename")
    print("3. Run this cell again")

## Step 8: Process Multiple Images in a Folder

In [ ]:
def process_folder(folder_path, output_folder="results"):
    """
    Process all images in a folder.
    """
    # Create output folder
    os.makedirs(output_folder, exist_ok=True)
    
    # Find images
    image_files = []
    for ext in ['.jpg', '.jpeg', '.png', '.tiff', '.tif', '.bmp']:
        image_files.extend([f for f in os.listdir(folder_path) if f.lower().endswith(ext)])
    
    if not image_files:
        print(f"No images found in {folder_path}")
        return
    
    print(f"Found {len(image_files)} images")
    print("="*50)
    
    # Process each image
    all_results = []
    
    for i, filename in enumerate(image_files, 1):
        print(f"\n[{i}/{len(image_files)}] {filename}")
        
        # Load
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        
        if img is None:
            print("  ❌ Could not load")
            continue
        
        # Detect
        defects = detector.detect(img)
        
        # Save result
        result_img = draw_defects(img, defects)
        output_path = os.path.join(output_folder, f"result_{filename}")
        cv2.imwrite(output_path, result_img)
        
        # Store results
        all_results.append({
            'filename': filename,
            'defect_count': len(defects),
            'defects': defects
        })
        
        print(f"  ✓ Found {len(defects)} defects")
    
    # Summary
    print("\n" + "="*50)
    print("BATCH PROCESSING COMPLETE")
    print("="*50)
    print(f"Processed: {len(all_results)} images")
    print(f"Total defects: {sum(r['defect_count'] for r in all_results)}")
    print(f"Results saved to: {output_folder}/")
    
    return all_results


# Example usage:
# results = process_folder("my_images_folder")

print("✓ Batch processing function ready!")
print("\nTo use: results = process_folder('your_folder_name')")

## Step 9: Adjust Detection Settings

Play with these settings to get better results!

In [ ]:
# ============================================
# ADJUST THESE SETTINGS
# ============================================

# Minimum defect size (pixels)
# Smaller = more sensitive (finds tiny defects but may find noise)
# Larger = less sensitive (only finds bigger defects)
MIN_SIZE = 10  # Try: 5, 10, 20, 30

# Maximum defect size (pixels)
# Prevents detecting very large regions
MAX_SIZE = 50000

# ============================================
# CREATE NEW DETECTOR WITH YOUR SETTINGS
# ============================================

custom_detector = SimpleDefectDetector(min_size=MIN_SIZE, max_size=MAX_SIZE)

# Test on synthetic image
custom_defects = custom_detector.detect(test_image)
show_results(test_image, custom_defects)

print("\n💡 TIP: If you see too many false positives, increase MIN_SIZE")
print("💡 TIP: If you're missing small defects, decrease MIN_SIZE")

## 🎯 Summary & Tips

### What This Notebook Does:
1. ✅ Finds scratches, voids, particles, and other defects
2. ✅ Works on any microscope image
3. ✅ Shows visual results with colored boxes
4. ✅ Can process single images or whole folders

### How to Use:
- **Test with synthetic image**: Run cells 1-6
- **Use your own image**: Edit and run cell 7
- **Process many images**: Edit and run cell 8
- **Adjust sensitivity**: Edit and run cell 9

### Color Legend:
- 🔴 **Red** = Scratches
- 🔵 **Blue** = Voids
- 🟢 **Green** = Particles
- 🟡 **Cyan** = Spots

### Troubleshooting:
- **Too many false positives?** → Increase `MIN_SIZE`
- **Missing small defects?** → Decrease `MIN_SIZE`
- **Image won't load?** → Check file path and format

### Need Help?
Just re-run any cell to try again!